## Import related functions

In [1]:
import cobra
import re 
import pandas as pd
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Input and output files

In [2]:
f = 0.45387051337830087  # 蛋白组计算

ptot = 0.605 # Z.mobilis PMID: 20967753 and PMID: 21092328
# The approximated average saturation of enzyme.
sigma = 0.5 # default
concentration = 40 # glucose uptake rate of q = 41.8 mmol/gdw/h   10.1016/j.ymben.2019.05.006
max_growth = 0.36  # "Jacobson et al.(2019)")
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3) 
EC_max_file='./data/EC_kcat_max.json'# https://www.brenda-enzymes.org/brenda_download/file_download.php
method='AutoPACMEN' #AutoPACMEN # AutoPACMEN_median # DLkcat # TurNup # UniKP
use_substrate='EX_glc_D_e'

obj='ZM_BIOMASS'

#Originl ecmodel and result file
ecModel_file="./model/iZM547_"+ method +".json"
fluxes_infile_ori = "./analysis/"+method+"ECMpy_solution_ZM_BIOMASS_pfba.csv"
reaction_kcat_MW_file = "./analysis/get_kcat_mw_by_%s/reaction_kcat_MW.csv"%method
need_change_reaction_list=[]
changed_reaction_list=[]
round_num=1
reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#ecmodel and result file
json_output_file = "./model/iZM516_20230711_irr_enz_constraint_adj_"+ method +".json"
reaction_kcat_MW_outfile = './analysis/get_kcat_mw_by_%s/reaction_change_by_enzuse.csv'%method

# Calibration kcat according Enzyme usage 

In [4]:
#Calibration
enz_model_obj=0
while (enz_model_obj < max_growth and round_num<50):#maxium growth rate
    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(ecModel_file,fluxes_infile_ori,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
    enz_model.objective=obj
    #change original substrate in model
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
            
    enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

    # enz_model_obj=enz_model.slim_optimize()
    enz_model_obj = cobra.flux_analysis.pfba(enz_model)
    print('Calibration round %s : '%round_num+str(enz_model_obj))
    round_num=round_num+1
    
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)